In [1]:
import numpy as np
import matplotlib.pyplot as plt
import torch
from torch import nn, optim

In [5]:
#Parameters
P = 1
snrdB = 6 
sigma2 = P/(10**(snrdB/10))

#Data
M = 4 # cardinality of the alphabet
chi = [-3., -1., 1., 3.] # alphabet
a = np.random.choice(M,10000) # messages

In [2]:
def normalization(x, P):
    c = torch.mean(x**2)
    return torch.sqrt(P / c) * x

In [3]:
def awgn(x, sigma2):
    noise_t = np.sqrt(sigma2)*torch.randn(x.shape)
    return torch.add(x, noise_t)

In [ ]:
def calculate_py_given_x(x, y):
    return (1/(np.sqrt(2*np.pi*sigma2))) * np.exp(-(y-x)**2/sigma2/2) 

In [4]:
#receive the logit from the CE loss function and correct with additional term
def loss_fn(logit, received_symbols, true_symbols):
    Q = nn.functional.softmax(logit, 1)
    c = mapper(true_symbols) #to-do: define the mapper
    term_1 = nn.CrossEntropyLoss(logit, c)
    term_2 = np.mean([np.sum(calculate_py_given_x(received_symbols, c_i)*np.log2(logit[c_i,:])) for c_i in c]) #is it correct to take the mean?

    return term_1 + term_2